In [232]:
import folium
import openrouteservice

from math import cos, asin, sqrt, isnan
import requests

import csv
import pandas as pd

# Dataset

  * [Demande de valeurs foncières](https://cadastre.data.gouv.fr/dvf#download)

In [519]:
house_dataset = pd.read_pickle('data/checkpoints/after_analyze.pkl.xz',  compression='xz')
house_dataset.head()

,date_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,nom_commune,surface_carrez,type_local,surface_reelle_bati,nombre_pieces_principales,surface_terrain,longitude,latitude
10,2014-01-02,194400.0,23,NaN,BD EMILE ROMANET,2794,44100,Nantes,NaN,Appartement,84.0,4,NaN,-1.603261,47.209692
12,2014-01-02,107000.0,11,NaN,RUE DU DOCTEUR ALFRED CORLAY,0882,44800,Saint-Herblain,45.80,Appartement,46.0,2,NaN,-1.644254,47.207462
14,2014-01-09,208154.0,38,NaN,RUE DE LA PLANCHE AU GUE,5555,44300,Nantes,103.64,Appartement,103.0,5,NaN,-1.519753,47.272364
17,2014-01-06,79000.0,8,NaN,RUE DES CARMELITES,1436,44000,Nantes,26.45,Appartement,25.0,1,NaN,-1.551322,47.216626
21,2014-01-02,335000.0,23,NaN,RUE DES CANARIS,1390,44300,Nantes,NaN,Maison,118.0,5,562.0,-1.506824,47.231122


In [399]:
# https://en.wikipedia.org/wiki/Haversine_formula
# Get distance between 2 points in km
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295 # Math.PI / 180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a)) # 2 * R; R = 6371 km

# Return N closest points
def closest(data, v, number):
    res = []
    for val in data:
        val['distance'] = distance(val['lat'],val['lon'],v['lat'],v['lon'])
    
    data.sort(key=lambda x: x['distance'])
    return data[:number]

# Return N closest points
def closestPanda(data, pandaRow, number):
    res = []
    for index, row in data.iterrows():
        res.append({'distance' : distance(row['latitude'],row['longitude'],pandaRow['latitude'],pandaRow['longitude']), 'longitude': row['longitude'], 'latitude': row['latitude']})
    
    res.sort(key=lambda x: x['distance'])
    return res[:number]

# Transform string to object 
# Example : '47.2079361916, -1.6101650945' -> {'lon': -1.6101650945, 'lat': 47.2079361916}
def strToCoordObj(val):
    tmp = val.split(',')
    return {'lon': float(tmp[1]), 'lat': float(tmp[0])}

# Transform Pandas.series string to list of coords
def dfToCoordList(df):
    df['longitude']
    return list(map(strToCoordObj , series.tolist()))

# Isochrone and POI retrieval

In [523]:
ORS_CLIENT = openrouteservice.Client(base_url='http://localhost:9090/ors')
POI_CLIENT = openrouteservice.Client(base_url='http://localhost:5000')

def addMarkerListToMap(coordList, current_map):
    for elem in coordList:
        folium.map.Marker([elem['latitude'],elem['longitude']], # reverse coords due to weird folium lat/lon syntax
                      icon=folium.Icon(color='red',
                                        icon_color='#cc0000',
                                        icon='pushpin',
                                        prefix='fa',
                                       ),
                 ).add_to(current_map) # Add apartment locations to map
    
    return current_map

def buildIsochrone(timeOrDistance, profile, longitude, latitude):
# Request of isochrones with 15 minute footwalk.
    params_iso = {'profile': profile,
                  'intervals': [timeOrDistance], # 900/60 = 15 mins
                  'attributes': ['total_pop'], # Get population count for isochrones
                  'locations': [[longitude,latitude]] # Add apartment coords to request parameters
             }    

    return ORS_CLIENT.isochrones(**params_iso) # Perform isochrone request

In [524]:
def getPOIForHouse(df, categories_poi, map1, index, timeOrDistance=900, profile='foot-walking'):
    # Retrieve specific house
    row = df.iloc[[index]]

    # Get Isochrones
    iso = buildIsochrone(timeOrDistance, profile,row['longitude'].item(), row['latitude'].item())
    
    # Draw isochrone on map
    folium.features.GeoJson(iso).add_to(map1)
    
    # Center map on house
    map1.location = [row['latitude'].item(),row['longitude'].item()]
    
    # Draw house marker on map
    folium.map.Marker(
        [row['latitude'].item(),row['longitude'].item()],
        icon=folium.Icon(color='lightgray', icon_color='#cc0000', icon='home', prefix='fa'),
        popup="%s %s\n %i m2,  %i pièces\n%i €" % (row['adresse_numero'].item(), row['adresse_nom_voie'].item(), row['surface_reelle_bati'].item(), row['nombre_pieces_principales'].item(),row['valeur_fonciere'].item())
    ).add_to(map1)
    
    # Common request parameters
    params_poi = {'request': 'pois', 'sortby': 'distance', 'geojson': iso['features'][0]['geometry'] }

    poi = {}
    treatedLocations = []

    for typ, category in categories_poi.items():
        params_poi['filter_category_ids'] = category
        poi[typ] = dict()
        poi[typ]['geojson'] = POI_CLIENT.places(**params_poi)['features'] # Actual POI request
        
        
        treatedLocations = []
        for elem in poi[typ]['geojson']:
            if elem['geometry']['coordinates'] not in treatedLocations:
                treatedLocations.append(elem['geometry']['coordinates'])
            
                folium.map.Marker(list(reversed(elem['geometry']['coordinates'])), # reverse coords due to weird folium lat/lon syntax
                          icon=folium.Icon(color='blue',
                                            icon_color='#cc0000',
                                            icon='pushpin',
                                            prefix='fa',
                                           ),
                     ).add_to(map1) # Add apartment locations to map
        poi[typ]['geojson'] = treatedLocations
        
        print("\t{}: {}".format(typ, len(poi[typ]['geojson'])))
        

    return map1    

In [525]:
def getPOIGroupForHouse(df, poi_group_id, map1, index, timeOrDistance=900, profile='foot-walking'):
    
     # Retrieve specific house
    row = df.iloc[[index]]

    # Get Isochrones
    iso = buildIsochrone(timeOrDistance, profile,row['longitude'].item(), row['latitude'].item())
    
    # Draw isochrone on map
    folium.features.GeoJson(iso).add_to(map1)
    
    # Center map on house
    map1.location = [row['latitude'].item(),row['longitude'].item()]
    
    # Draw house marker on map
    folium.map.Marker(
        [row['latitude'].item(),row['longitude'].item()],
        icon=folium.Icon(color='lightgray', icon_color='#cc0000', icon='home', prefix='fa'),
        popup="%s %s\n %i m2,  %i pièces\n%i €" % (row['adresse_numero'].item(), row['adresse_nom_voie'].item(), row['surface_reelle_bati'].item(), row['nombre_pieces_principales'].item(),row['valeur_fonciere'].item())
    ).add_to(map1)
    
    
    # Common request parameters
    params_poi = {'request': 'pois', 'filter_category_group_ids': poi_group_id, 'sortby': 'distance', 'geojson': iso['features'][0]['geometry'] }
    
    poi = {}
    treatedLocations = []
    
    poi = POI_CLIENT.places(**params_poi)['features'] # Actual POI request
    treatedLocations = []
    for elem in poi:
        if elem['geometry']['coordinates'] not in treatedLocations:
            treatedLocations.append(elem['geometry']['coordinates'])
            
            folium.map.Marker(list(reversed(elem['geometry']['coordinates'])), # reverse coords due to weird folium lat/lon syntax
                              icon=folium.Icon(color='blue',
                                               icon_color='#cc0000',
                                               icon='pushpin',
                                               prefix='fa',
                                              ),
                             ).add_to(map1) # Add apartment locations to map
    poi = treatedLocations
        
    print("\t{}: {}".format(typ, len(poi)))
        
    return map1

# Visualization

In [520]:
map1 = folium.Map(tiles='openstreetmap', location=([47.218371, -1.553621]), zoom_start=12)

categories_poi = {
        'hospital':[206],
    }

poiForHouse(house_dataset, categories_poi, map1, 3)

	hospital: 5


In [505]:
map1 = folium.Map(tiles='openstreetmap', location=([47.218371, -1.553621]), zoom_start=12)

getPOIGroupForHouse(house_dataset, [800], map1, 3)

	university: 132


# Data embedding  
On identifie 5 catégories distinctes. Les nombres associés à chaque catégories ou points d'intérêts est l'identifiant unique.
Celui-ci permet de récupérer les informations via l'API

 * Les transports -> 800
   * arrêt de bus -> 801  
   * arrêt de tram -> 802
   * gare SNCF -> 803
   
* Les parcs -> 810
  * parc -> 811
  * jeu pour enfant -> 812
  
* La vie courante -> 820
  * supermarché -> 821
  * épicier -> 822
  * mairie -> 823
  * pharmacie -> 824
  * établissement social -> 825
  
* Les distractions  -> 830
  * zoo  -> 831
  * gallerie d'art  -> 832
  * musée  -> 833
  * librairie  -> 834
  * centre artistique  -> 835
  * restaurant  -> 836
  * bar  -> 837
  * lieux de culte  -> 838

* Les soins -> 840
  * docteurs -> 841
  * clinic -> 842
  * hospital -> 843